In [3]:
import pandas as pd

In [5]:
country = "Peru"
year = 2021
election_type = "lower"
aggregation = "precinct"

In [19]:
df = pd.read_csv(f"data_output/{country}/{year}_{election_type}_{aggregation}.csv.gz")
print(df.shape)
df = df[df["candidate"].str.strip() != ""]
print(df.shape)
df = df[~df["candidate"].isna()]
print(df.shape)

df.head()

(2720, 6)
(2325, 6)
(2301, 6)


,candidate_id,candidate,position,value,district_id,party_id
0,129.0,EDILBERTO KUJANCHAM PETSA,1,846,D44001,6
1,130.0,MARIA LUCELINA MORETO COPIA,2,119,D44001,6
2,131.0,ERNESTOR VASQUEZ CABRERA,3,704,D44001,6
3,132.0,EULA ONIRIA DAVILA DAVILA,4,12,D44001,6
4,757.0,HENRY GERMAN CHICO LEON,1,7134,D44002,6


In [20]:
coalition = df.groupby(["district_id", "party_id"]).agg({"value": "sum", "candidate": "nunique"}).rename(columns={"candidate": "Q", "value": "N"}).reset_index()
coalition.head()

,district_id,party_id,N,Q
0,D44001,1,6633,4
1,D44001,2,6155,4
2,D44001,4,149,4
3,D44001,5,1850,4
4,D44001,6,1681,4


In [21]:
data = pd.merge(df, coalition, on=["district_id", "party_id"])
data["vQ/N"] = data["value"] * data["Q"] / data["N"]
data.head()

,candidate_id,candidate,position,value,district_id,party_id,N,Q,vQ/N
0,129.0,EDILBERTO KUJANCHAM PETSA,1,846,D44001,6,1681,4,2.013087
1,130.0,MARIA LUCELINA MORETO COPIA,2,119,D44001,6,1681,4,0.283165
2,131.0,ERNESTOR VASQUEZ CABRERA,3,704,D44001,6,1681,4,1.675193
3,132.0,EULA ONIRIA DAVILA DAVILA,4,12,D44001,6,1681,4,0.028554
4,757.0,HENRY GERMAN CHICO LEON,1,7134,D44002,6,15545,6,2.753554


In [22]:
data.to_csv(f"data_universality/{country}_{year}_{election_type}.csv", index=False)